<a href="https://colab.research.google.com/github/RenzoZanca/Numeric_Proyects/blob/main/optimizacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tarea numérica optimización

Renzo Zanca Ruedlinger

Importamos las librerias necesarias:

In [ ]:
import numpy as np
import scipy.optimize
import time

## P1


**Creamos la matriz de categorias considerando los siguientes correos:**


**Correo 1:** Correo de promoción de Uber, largo mediano, en español:

$x_1^1 = 1, x_1^2 = 2, x_1^3 = 2, y_1 = 1$

**Correo 2:** Correo institucional de la universidad, en español, largo:

$x_2^1 = 1, x_2^2 = 2, x_2^3 = 3, y_2 = 0$

**Correo 3:** Correo de un amigo, en español, largo mediano:

$x_3^1 = 1, x_3^2 = 1, x_3^3 = 2, y_3 = 0$

**Correo 4:** Correo de un desconocido, en ingles, largo:

$x_4^1 = 2, x_4^2 = 1, x_4^3 = 3, y_4 = 1$

**Correo 5:** Correo de admisión universidad Santo Tomás, largo, en español:

$x_5^1 = 1, x_5^2 = 2, x_5^3 = 3, y_5 = 1$

**Correo 6:** correo de servidor extraño, en español, extenso, de mafioso italiano:

$x_6^1 = 1, x_6^2 = 3, x_6^3 = 3, y_6 = 1$

**Correo 7:** Correo de servidor extraño, promocionando cremas para pies, largo mediano, en español:

$x_7^1 = 1, x_7^2 = 3, x_7^3 = 2, y_7 = 1$

**Correo 8:** Correo univesitario corto, en español:

$x_8^1 = 1, x_8^2 = 2, x_8^3 = 1, y_8 = 0$

**Correo 9:** Correo de servidor extraño ofreciendo bitcoins gratis, en ingles, largo mediano:

$x_9^1 = 2, x_9^2 = 3, x_9^3 = 2, y_9 = 1$

**Correo 10:** Correo corto del banco, en español:

$x_{10}^1 = 1, x_{10}^2 = 2, x_{10}^3 = 1, y_{10} = 0$

**Correo 11:** Correo de compañero de universidad, corto, en español:

$x_{11}^1 = 1, x_{11}^2 = 1, x_{11}^3 = 1, y_{11} = 0$

**Correo 12:** Correo de empresa francesa, largo mediano, en frances:

$x_{12}^1 = 3, x_{12}^2 = 2, x_{12}^3 = 2, y_{12} = 1$

El criterio para determinar si un correo es spam o no es simplemente mi criterio personal, pues yo soy el usuario del correo.

Lo pasamos a código creando matrices y vectores con numpy:

In [ ]:
#creamos la matriz de categorias:
x = np.zeros(shape=(3,12), dtype='int')
#rellenamos con los datos:
x[0] = [1,1,1,2,1,1,1,1,2,1,1,3] #x^1
x[1] = [2,2,1,1,2,3,3,2,3,2,1,2] #x^2
x[2] = [2,3,2,3,3,3,2,1,2,1,1,2] #x^3

#creamos el vector de respuestas:
y = np.array([1,0,0,1,1,1,1,0,1,0,0,1])

## P2

In [ ]:
#Variables globales:
n = 12
k = 3

### Función de pérdida de entropía cruzada:

Para implementar esta función, se asume que la dimension de b sera igual a la de $x_i^T$, pues en la práctica siempre se cumplirá cuando se utilice. Con esto la función recibe un vector b y devuelve un arreglo con un único elemento r(b). En el código se utiliza la función np.dot(), la cual sirve para hacer productos puntos, sin importar si el vector está traspuesto o no (obtiene el mismo resultado). El algoritmo consiste en inicializar con un arreglo con un cero y luego ir sumando iterativamente cada elemento de la sumatoria.

In [ ]:
def r(b):
  suma = np.zeros(1, dtype='float')
  for j in range(n):
    suma += ((1-y[j])*np.dot(x[:,j],b) + np.log(1 + np.exp(-np.dot(x[:,j],b))))

  return suma/n

### Gradiente de la función de pérdida de entropía cruzada

La función recibe un vector b y devuelve un arreglo con el gradiente de la función r. Como función auxiliar, se define la funcion h, la cual aparece en la expresión del gradiente; se asumira que recibe un valor escalar, pues en la práctica siempre será asi. De forma similar, el algoritmo consiste en inicializar con un arreglo con ceros y luego ir sumando iterativamente cada elemento de la sumatoria.

In [ ]:
def h(z):
  return 1/(1 + np.exp(-z))

def gradf(b):
  suma = np.zeros(k, dtype='float')
  for j in range(n):
    suma += (np.dot((h(np.dot(x[:,j],b)) - y[j]), x[:,j]))

  return suma/n

### Matriz hessiana de la función de pérdida de entropía cruzada


La función recibe un vector b y devuelve un arreglo con la matriz hessiana de la función r. Se utilizara la función np.multiply() para la multiplicación de matrices. Para obtener el vector $x_i^T$ se pasa el vector x a una matriz (encerrandolo con corchetes) y luego se llama al metodo .T que transpone la matriz. De igual manera, el algoritmo consiste en inicializar con un arreglo con ceros y luego ir sumando iterativamente cada elemento de la sumatoria.

In [ ]:
def hessf(b):
  suma = np.zeros(shape=(3,3), dtype='float')
  for j in range(n):
    suma += (h(np.dot(x[:,j],b)) * ((1-h(np.dot(x[:,j],b))) * np.multiply(x[:,j], np.array([x[:,j]]).T)))

  return suma/n

### Función busqueda exacta




Esta función recibe un punto b0 y una dirreción d, devolviendo el paso de una busqueda lineal exacta. Para buscar el paso a, debemos minimizar phi = r(b0 + ad), para ello usamos la función minimize_scalar de la libreria scipy.optimize.

In [ ]:
def busquedaexacta(b0,d):
  #definimos phi:
  def phi(a):
    return r(b0 + a*d)
  a = scipy.optimize.minimize_scalar(phi)
  return a.x[0]

### Función backtracking

Esta función recibe un punto b0 y una dirreción d, devolviendo el paso al utilizar backtracking. Para buscar el paso a, usamos el algoritmo visto en clases auxiliares (resumen auxiliar 11). Consideramos un valor de p = 0.5

In [ ]:
def backtracking(bo,d):
  a_barra = 1
  p = 0.5
  c = 0.9
  a = a_barra
  while True:
    if r(bo + a*d)[0] <= r(bo)[0] + c*a*np.dot(np.transpose(gradf(bo)),d):
      break
    else:
      a = p*a

  return a

### Función método gradiente

Esta función recibe un punto b0 y un str con el tipo de paso que se debe usar. Retorna una lista con el punto óptimo b, su valor optimo robj y el numero de iteraciones que realiza el algoritmo. 

Como función auxiliar, definimos la norma euclideana. El algoritmo consitira en iniciar con b = b0, para luego ir actualizando el valor de b siguiendo la regla b = b + ad, en donde d es -gradf(b) y a el paso, que puede ser de busqueda exacta, backtracking o de line_search de scipy.optimize. El algoritmo se detiene cuando la norma de gradf(b) es menor a 0.00001 o cuando ocurren mas de 100000 iteraciones.

In [ ]:
def norma(z):
  l = len(z)
  suma = 0
  for i in range(l):
    suma += (z[i] ** 2)
  return np.sqrt(suma)

#recibe un punto inicial bo
#y un str con el tipo de paso que se debe usar
#retorna una lista con el punto optimo b, su valor
#optimo robj y el numero de iteraciones
def MetodoGradiente(b0,pasos):

  #busqueda exacta:
  if pasos == 'be':
    iter = 0 #numero de iteraciones
    b = b0 #punto optimo 
    while iter < 100000: #max de iteraciones
      #verificamos si la norma del gradiente es menor que epsilon
      if norma(gradf(b)) < 0.00001:
        iter += 1
        break
        
      d = -gradf(b) #dirreccion de descenso
      a = busquedaexacta(b,d) #paso
      b = b + a*d #actualizamos el valor de b
      iter += 1

    robj = r(b)
    return [b,robj,iter]
  

  #backtracking:
  elif pasos == 'bt':
    iter = 0 #numero de iteraciones
    b = b0 #punto optimo 
    while iter < 100000: #max de iteraciones
      #verificamos si la norma del gradiente es menor que epsilon
      if norma(gradf(b)) < 0.00001:
        iter += 1
        break
        
      d = -gradf(b) #direccion de descenso
      a = backtracking(b,d) #paso
      b = b + a*d #actualizamos el valor de b
      iter += 1

    robj = r(b)
    return [b,robj,iter]


  #optimize:
  elif pasos == 'o':
    iter = 0 #numero de iteraciones
    b = b0 #punto optimo 
    while iter < 100000: #max de iteraciones
      #verificamos si la norma del gradiente es menor que epsilon
      if norma(gradf(b)) < 0.00001:
        iter += 1
        break
        
      d = -gradf(b) #direccion de descenso
      a =  scipy.optimize.line_search(r,gradf,b,d)[0] #paso
      #caso donde no existe alpha:
      if a == None:
        return 'metodo no converge'
      b = b + a*d #actualizamos el valor de b
      iter += 1

    robj = r(b)
    return [b,robj,iter]


  else:
    return 'paso no existente'

### Función método de Newton

De forma casi análoga, esta función recibe un punto b0 y un str con el tipo de paso que se debe usar. Retorna una lista con el punto óptimo b, su valor optimo robj y el numero de iteraciones que realiza el algoritmo.

 El algoritmo consitira en iniciar con b = b0, para luego ir actualizando el valor de b siguiendo la regla b = b + ad, en donde d es el producto entre -gradf(b) y la inversa de hessf(b) y a el paso, que puede ser de busqueda exacta, backtracking o de line_search de scipy.optimize. El algoritmo se detiene cuando la norma de gradf es menor a 0.00001 o cuando ocurren mas de 100000 iteraciones.

In [ ]:
def MetodoNewton(b0,pasos):

  #busqueda exacta:
  if pasos == 'be':
    iter = 0 #numero de iteraciones
    b = b0 #punto optimo 
    while iter < 100000: #max de iteraciones
      #verificamos si la norma del gradiente es menor que epsilon
      if norma(gradf(b)) < 0.00001:
        iter += 1
        break

      hinv = np.linalg.inv(hessf(b))  #inversa del heassiano
      d = -np.dot(hinv,gradf(b)) #dirreccion de descenso
      a = busquedaexacta(b,d) #paso
      b = b + a*d #actualizamos el valor de b
      iter += 1

    robj = r(b)
    return [b,robj,iter]
  

  #backtracking:
  elif pasos == 'bt':
    iter = 0 #numero de iteraciones
    b = b0 #punto optimo 
    while iter < 100000: #max de iteraciones
      #verificamos si la norma del gradiente es menor que epsilon
      if norma(gradf(b)) < 0.00001:
        iter += 1
        break
        
      hinv = np.linalg.inv(hessf(b))  #inversa del heassiano
      d = -np.dot(hinv,gradf(b)) #dirreccion de descenso
      a = backtracking(b,d) #paso
      b = b + a*d #actualizamos el valor de b
      iter += 1

    robj = r(b)
    return [b,robj,iter]


  #optimize:
  elif pasos == 'o':
    iter = 0 #numero de iteraciones
    b = b0 #punto optimo 
    while iter < 100000: #max de iteraciones
      #verificamos si la norma del gradiente es menor que epsilon
      if norma(gradf(b)) < 0.00001:
        iter += 1
        break
        
      hinv = np.linalg.inv(hessf(b))  #inversa del heassiano
      d = -np.dot(hinv,gradf(b)) #dirreccion de descenso
      a =  scipy.optimize.line_search(r,gradf,b,d)[0] #paso
      #caso donde no existe alpha:
      if a == None:
        return 'metodo no converge'
      b = b + a*d #actualizamos el valor de b
      iter += 1

    robj = r(b)
    return [b,robj,iter]


  else:
    return 'paso no existente'

## P3

Para rellenar la tabla, primero debemos testear los métodos, calculando sus valores y midiendo el tiempo que se demoran utilizando la libreria time.

In [ ]:
b1 = np.array([1,1,1]) #punto inicial

In [ ]:
#gradiente busqueda exacta:

t1 = time.time()
print('gradiente busqueda exacta: ')
print('')
resp = MetodoGradiente(b1,'be')
print('b óptimo: ', resp[0])
print('valor óptimo: ', resp[1])
print('número de iteraciones: ', resp[2])
t2 = time.time()
print('tiempo: ', t2-t1, 'segundos')

gradiente busqueda exacta: 

b óptimo:  [ 0.30822855 -0.04522163  0.18509475]
valor óptimo:  [0.62960812]
número de iteraciones:  26
tiempo:  0.14033770561218262 segundos


In [ ]:
#gradiente backtracking:
t1 = time.time()
print('gradiente backtracking: ')
print('')
resp = MetodoGradiente(b1,'bt')
print('b óptimo: ', resp[0])
print('valor óptimo: ', resp[1])
print('número de iteraciones: ', resp[2])
t2 = time.time()
print('tiempo: ', t2-t1, 'segundos')

gradiente backtracking: 

b óptimo:  [ 0.30822464 -0.04519791  0.18507516]
valor óptimo:  [0.62960812]
número de iteraciones:  123
tiempo:  0.15927696228027344 segundos


In [ ]:
#gradiente line search optimize:

t1 = time.time()
print('gradiente line search optimize: ')
print('')
resp = MetodoGradiente(b1,'o')
print('b óptimo: ', resp[0])
print('valor óptimo: ', resp[1])
print('número de iteraciones: ', resp[2])
t2 = time.time()
print('tiempo: ', t2-t1, 'segundos')

gradiente line search optimize: 

b óptimo:  [ 0.30818755 -0.04519713  0.18509919]
valor óptimo:  [0.62960812]
número de iteraciones:  104
tiempo:  0.12311673164367676 segundos


In [ ]:
#newton busqueda exacta:

t1 = time.time()
print('Newton busqueda exacta: ')
print('')
resp = MetodoNewton(b1,'be')
print('b óptimo: ', resp[0])
print('valor óptimo: ', resp[1])
print('número de iteraciones: ', resp[2])
t2 = time.time()
print('tiempo: ', t2-t1, 'segundos')
print('')

Newton busqueda exacta: 

b óptimo:  [ 0.30811787 -0.04519188  0.18513565]
valor óptimo:  [0.62960812]
número de iteraciones:  5
tiempo:  0.05759716033935547 segundos



In [ ]:
#newton backtracking:

t1 = time.time()
print('Newton backtracking: ')
print('')
resp = MetodoNewton(b1,'bt')
print('b óptimo: ', resp[0])
print('valor óptimo: ', resp[1])
print('número de iteraciones: ', resp[2])
t2 = time.time()
print('tiempo: ', t2-t1, 'segundos')
print('')

Newton backtracking: 

b óptimo:  [ 0.30812142 -0.04518976  0.18513744]
valor óptimo:  [0.62960812]
número de iteraciones:  87
tiempo:  0.4374406337738037 segundos



In [ ]:
#newton line search optimize:

t1 = time.time()
print('Newton line search optimize: ')
print('')
resp = MetodoNewton(b1,'o')
print('b óptimo: ', resp[0])
print('valor óptimo: ', resp[1])
print('número de iteraciones: ', resp[2])
t2 = time.time()
print('tiempo: ', t2-t1, 'segundos')
print('')

Newton line search optimize: 

b óptimo:  [ 0.3080962  -0.04519037  0.18514301]
valor óptimo:  [0.62960812]
número de iteraciones:  6
tiempo:  0.03347182273864746 segundos



Con esto rellenamos los datos en una tabla:

$\begin{array}{|c|c|} \hline
\text{Método} & Paso & \text{Punto inicial} & \text{Óptimo} & \text{Valor objetivo} & Iteraciones & Tiempo [s] \\ \hline
Gradiente       & \text{Búsqueda exacta}         & (1,1,1)                                                          & (0.30822, -0.04522,  0.18509)                                        & 0.62960812                                                         & 26                                                                    & 0.13064                 \\ \hline
Gradiente       & Backtracking            & (1,1,1)                                                          & (0.30822, -0.04519,  0.18507)                                         & 0.62960812                                                        & 123                                                                   & 0.15126                 \\ \hline
Gradiente       & \text{Line search de optimize} & (1,1,1)                                                          & (0.30818, -0.04519,  0.18509)                                        & 0.62960812                                                          & 104                                                                  & 0.09818                 \\ \hline
Newton          & \text{Búsqueda exacta}         & (1,1,1)                                                          & (0.30811, -0.04519,  0.18513)                                        & 0.62960812                                                          & 5                                                                     & 0.04390                 \\ \hline
Newton          & Backtracking            & (1,1,1)                                                          & (0.30812, -0.04518,  0.18513)                                         & 0.62960812                                                         & 87                                                                    & 0.37921                 \\ \hline
Newton          & \text{Line search de optimize} & (1,1,1)                                                          & (0.30809, -0.04519,  0.18514)                                         & 0.62960812                                                         & 6                                                                     & 0.02201                 \\ \hline
\end{array}$

Como observación, los tiempos pueden variar para distintas ejecuciones del código.

## P4

Notamos que todos los métodos llegaron al mismo valor óptimo, salvo pequeñas diferencias decimales. Luego, para decidir cual es el mejor método debemos considerar la eficiencia de estos. Notamos que el método de Newton con paso de line search de optimize obtiene el menor tiempo y está a una repetición del menor numero de repeticiones, por lo que lo consideramos como el mejor método para este ejemplo.

Ahora probemos este método con otros puntos iniciales de igual manera que lo hicimos en la parte anterior:

In [ ]:
b2 = (-1,-1,-1)
b3 = (10,10,10)
b4 = (-1,0,1)

In [ ]:
#b2

t1 = time.time()
print('punto inicial b2: ')
print('')
resp = MetodoNewton(b2,'o')
print('b óptimo: ', resp[0])
print('valor óptimo: ', resp[1])
print('número de iteraciones: ', resp[2])
t2 = time.time()
print('tiempo: ', t2-t1, 'segundos')
print('')

punto inicial b2: 

b óptimo:  [ 0.30811874 -0.04519347  0.18513605]
valor óptimo:  [0.62960812]
número de iteraciones:  7
tiempo:  0.02793264389038086 segundos



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: overflow encountered in exp
  after removing the cwd from sys.path.


Para la dirrección b3 notamos que su hessiano es una matriz singular (determinante = 0), luego su inversa no se puede calcular y el método de Newton no nos sirve:

In [ ]:
print(hessf(b3))
det = np.linalg.det(hessf(b3))
print('Determinante: ', det)

[[7.79006489e-15 7.79006489e-15 7.79006489e-15]
 [7.79006489e-15 7.79006489e-15 7.79006489e-15]
 [7.79006489e-15 7.79006489e-15 7.79006489e-15]]
Determinante:  0.0


In [ ]:
#b4

t1 = time.time()
print('punto inicial b4: ')
print('')
resp = MetodoNewton(b4,'o')
print('b óptimo: ', resp[0])
print('valor óptimo: ', resp[1])
print('número de iteraciones: ', resp[2])
t2 = time.time()
print('tiempo: ', t2-t1, 'segundos')
print('')

punto inicial b4: 

b óptimo:  [ 0.30811933 -0.04519369  0.185136  ]
valor óptimo:  [0.62960812]
número de iteraciones:  5
tiempo:  0.01907515525817871 segundos



## P5

Al ver los resultados de la parte anterior, notamos que todos obtienen el mismo valor óptimo, salvo pequeñas diferencias decimales, luego compararemos basandonos en su eficiencia. Anotamos los resultados anotándolos en una tabla:

$\begin{array}{|c|c|} \hline
\text{Punto inicial} & Iteraciones & \text{Tiempo [s]} \\ \hline
b1 & 6 & 0.02201 \\ \hline
b2 & 7 & 0.02977 \\\hline
b3 & - & -  \\ \hline
b4 & 5 & 0.01963 \\ \hline
\end{array}$

Al tener el menor tiempo y el menor numero de iteraciones, consideramos al punto b4 = (-1,0,1) como el mejor punto inicial. Concluimos entonces que el valor óptimo de b es b = (0.30811933, -0.04519369, 0.185136)

Ahora analicemos los correos de test:

**Correo test 1:** 

Es un correo en español, de un servidor institucional, tiene menos de 50 palabras y no es considerado como spam. Con esto:

$xt_1^1 = 1, xt_1^2 = 2, xt_1^3 = 1, yt_1 = 0$

**Correo test 2:** 

Es un correo en ingles, de un servidor conocido (gmail), tiene mas de 100 palabras y es considerado como spam. Con esto:

$xt_2^1 = 2, xt_2^2 = 1, xt_2^3 = 3, yt_2 = 1$

**Correo test 3:** 

Es un correo en español, de un servidor desconocido, tiene mas de 100 palabras y es considerado como spam. Con esto:

$xt_3^1 = 1, xt_3^2 = 3, xt_3^3 = 3, yt_3 = 1$

Escribimos los xt en forma vectorial:

In [ ]:
xt1 = np.array([1,2,1])
xt2 = np.array([2,1,3])
xt3 = np.array([1,3,3])

Finalmente, para realizar las predicciones, calculamo el valor al realizar la regresión, osea calcular h(x_test * b_optimo):

In [ ]:
bopt = np.array([0.30811933, -0.04519369, 0.185136]) #optimo encontrado
ypred1 = h(np.dot(xt1,bopt))
ypred2 = h(np.dot(xt2,bopt))
ypred3 = h(np.dot(xt3,bopt))
print('Predicción correo 1: ', ypred1)
print('Predicción correo 2: ', ypred2)
print('Predicción correo 3: ', ypred3)

Predicción correo 1:  0.5993765204770101
Predicción correo 2:  0.7551837133996202
Predicción correo 3:  0.6743544317804203


Anotamos los resultados en una tabla:

$\begin{array}{|c|c|} \hline
 & \text{prediccion} & \text{valor esperado} \\ \hline
\text{correo 1} & 0.5993 & 0 \\ \hline
\text{correo 2} & 0.7551 & 1 \\\hline
\text{correo 3} & 0.6743 & 1  \\ \hline
\end{array}$

Notamos que los valores de la predicción se alejan un poco  de lo esperado. El correo 1 deberia ser mas cercano a 0, sin embargo se acerca a 0.6, esto se puede deber a que en mi perspectiva, esto si es un correo de spam, por lo que mi matriz de categorias no se acerca a lo planteado en el enunciado. Los correos 2 y 3 estan mas cerca de 1 que 0, sin embargo, son un poco lejanos.

Probablemente, los resultados podrian mejorar si se considera una matriz de categorias más grande (entrenar con más ejemplos) o si se consideran más variables sobre el correo.